In [1]:
#### Libraries

import pandas as pd
import numpy as np

# Visualization - Libraries

import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.models import CategoricalColorMapper
#from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6

output_notebook()

Loading BokehJS ...

In [2]:
# cov_df : Has cumulative count of covid for different states in US
path = r"C:\Users\LOCALGHOST\Desktop\USA\data\US_cases_covid.csv" #add your path here"
us_st_cov = pd.read_csv(path)

# To add latitude and longitude of different states
path = r"C:\Users\LOCALGHOST\Desktop\USA\data\us_statelatlong.csv" #add your path here 
lat_long_df = pd.read_csv(path, encoding = 'iso-8859-1',)

# reg_df : Has regions for different states in US
path = r"C:\Users\LOCALGHOST\Desktop\USA\data\us_regions.csv" #add your path here"
reg_df = pd.read_csv(path)

# fips_df : Has fips for different states in US
path = r"C:\Users\LOCALGHOST\Desktop\USA\data\us_state_fips.csv" #add your path here"
fips_df = pd.read_csv(path)

# pop_df : Has fips for different states in US
path = r"C:\Users\LOCALGHOST\Desktop\USA\data\us_pop.csv" #add your path here"
pop_df = pd.read_csv(path)

#### Merge Df's

In [3]:
display(us_st_cov.head(),us_st_cov.shape,lat_long_df.head(),lat_long_df.shape,reg_df.head(),reg_df.shape,fips_df.head(),fips_df.shape)

,Location,Confirmed,Cpmp,Recovered,Deaths
0,New York,"3,97,684","20,392","70,435","31,143"
1,California,"2,23,995","5,944",No data,"5,979"
2,New Jersey,"1,73,126","19,626","30,163","14,992"
3,Texas,"1,58,756","5,732","81,335","2,430"
4,Florida,"1,46,333","7,678",No data,"3,446"


(56, 5)

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


(51, 4)

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


(51, 4)

,stname,st,stusps
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


(51, 3)

In [4]:
list1 = list(reg_df.State)
list2 = list(lat_long_df.City)
cov_st_list = list(us_st_cov.Location)

Check if all elements are present 

In [5]:
result =  all(elem in list1  for elem in list2)
if result:
    print("Yes, list1 contains all elements in list2")    
else :
    print("No, list1 does not contains all elements in list2")

Yes, list1 contains all elements in list2


In [6]:
print(np.setdiff1d(list2,list1))

[]


In [7]:
# cov_df has some elements that are not in the other two df's. Check !

main_list = np.setdiff1d(cov_st_list,list1)

In [8]:
main_list

array(['American Samoa', 'Guam', 'Northern Mariana Islands',
       'Puerto Rico', 'U.S. Virgin Islands'], dtype='<U24')

These are Territories of the United States. Lets ignore that for now.

In [9]:
us_st_cov = us_st_cov[~us_st_cov['Location'].isin(main_list)]

Check for duplicate values

In [10]:
us_st_cov[us_st_cov.duplicated()]

,Location,Confirmed,Cpmp,Recovered,Deaths


In [11]:
lat_long_df[lat_long_df.duplicated()]

,State,Latitude,Longitude,City


In [12]:
reg_df[reg_df.duplicated()]

,State,State Code,Region,Division


No duplicates found.

In [13]:
display(lat_long_df.head(),lat_long_df.shape,reg_df.head(),reg_df.shape)

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


(51, 4)

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


(51, 4)

In [14]:
merged = pd.merge(lat_long_df, reg_df, left_on = "State", right_on = "State Code", how = "inner" )

In [15]:
merged.shape

(51, 8)

In [16]:
merged.head()

,State_x,Latitude,Longitude,City,State_y,State Code,Region,Division
0,AL,32.601011,-86.680736,Alabama,Alabama,AL,South,East South Central
1,AK,61.302501,-158.775020,Alaska,Alaska,AK,West,Pacific
2,AZ,34.168219,-111.930907,Arizona,Arizona,AZ,West,Mountain
3,AR,34.751928,-92.131378,Arkansas,Arkansas,AR,South,West South Central
4,CA,37.271875,-119.270415,California,California,CA,West,Pacific


In [17]:
merged = merged.rename(columns = {'State_y':'State'}).drop(columns=["State_x",'City'])
display(merged.head(),fips_df.head())

,Latitude,Longitude,State,State Code,Region,Division
0,32.601011,-86.680736,Alabama,AL,South,East South Central
1,61.302501,-158.775020,Alaska,AK,West,Pacific
2,34.168219,-111.930907,Arizona,AZ,West,Mountain
3,34.751928,-92.131378,Arkansas,AR,South,West South Central
4,37.271875,-119.270415,California,CA,West,Pacific


,stname,st,stusps
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


In [18]:
fips_df[fips_df.stname.duplicated()]

,stname,st,stusps


check if all items are present

In [19]:
result =  all(elem in list(merged['State'].unique())  for elem in list(fips_df['stname']))
if result:
    print("Yes, list1 contains all elements in list2")    
else :
    print("No, list1 does not contains all elements in list2")

Yes, list1 contains all elements in list2


In [20]:
merged = pd.merge(merged, fips_df, left_on= "State", right_on = "stname",how ="inner")

In [21]:
merged.columns = merged.columns.str.strip()

In [22]:
merged.head()

,Latitude,Longitude,State,State Code,Region,Division,stname,st,stusps
0,32.601011,-86.680736,Alabama,AL,South,East South Central,Alabama,1,AL
1,61.302501,-158.775020,Alaska,AK,West,Pacific,Alaska,2,AK
2,34.168219,-111.930907,Arizona,AZ,West,Mountain,Arizona,4,AZ
3,34.751928,-92.131378,Arkansas,AR,South,West South Central,Arkansas,5,AR
4,37.271875,-119.270415,California,CA,West,Pacific,California,6,CA


In [23]:
merged = merged.rename(columns = {"State Code":"StateCode","st":"Fips"}).drop(columns = ["stusps","stname"])

******************* 

Merging with population data

***** 

In [24]:
display(merged.head(2),merged.shape,pop_df.head(2),pop_df.shape)

,Latitude,Longitude,State,StateCode,Region,Division,Fips
0,32.601011,-86.680736,Alabama,AL,South,East South Central,1
1,61.302501,-158.775020,Alaska,AK,West,Pacific,2


(51, 7)

,Pop_Rank,State,Est_Pop_2019,Perc_of_Tot_Pop
0,1,California,39512223,0.1191
1,2,Texas,28995881,0.0874


(51, 4)

In [25]:
pop_df[pop_df.State.duplicated()]

,Pop_Rank,State,Est_Pop_2019,Perc_of_Tot_Pop


check if all items are present

In [26]:
result =  all(elem in list(merged['State'].unique())  for elem in list(pop_df['State'].unique()))
if result:
    print("Yes, list1 contains all elements in list2")    
else :
    print("No, list1 does not contains all elements in list2")

No, list1 does not contains all elements in list2


Problem in data : Cross check and rectify / Alternatively we can correct the CSV file

In [27]:
pop_df[~(pop_df.State.isin(merged.State))]

,Pop_Rank,State,Est_Pop_2019,Perc_of_Tot_Pop
48,49,DC,705749,0.0021


In [28]:
merged[~(merged.State.isin(pop_df.State))] # in pop table we hav abbr for Distric of Columbia

,Latitude,Longitude,State,StateCode,Region,Division,Fips
8,38.899349,-77.014567,District of Columbia,DC,South,South Atlantic,11


In [29]:
pop_df[pop_df['State']=='DC'].index #check index of problematic row

Int64Index([48], dtype='int64')

In [30]:
pop_df.loc[48,'State'] = "District of Columbia" # changed value

check if all items are present

In [31]:
result =  all(elem in list(merged['State'].unique())  for elem in list(pop_df['State'].unique()))
if result:
    print("Yes, list1 contains all elements in list2")    
else :
    print("No, list1 does not contains all elements in list2")

Yes, list1 contains all elements in list2


In [32]:
pop_merged = pd.merge(merged, pop_df, left_on= "State", right_on = "State",how ="inner")

In [33]:
pop_merged.head()

,Latitude,Longitude,State,StateCode,Region,Division,Fips,Pop_Rank,Est_Pop_2019,Perc_of_Tot_Pop
0,32.601011,-86.680736,Alabama,AL,South,East South Central,1,24,4903185,0.0148
1,61.302501,-158.775020,Alaska,AK,West,Pacific,2,48,731545,0.0022
2,34.168219,-111.930907,Arizona,AZ,West,Mountain,4,14,7278717,0.0219
3,34.751928,-92.131378,Arkansas,AR,South,West South Central,5,33,3017825,0.0091
4,37.271875,-119.270415,California,CA,West,Pacific,6,1,39512223,0.1191


********* 

All essential merges completed !!!!! 

******** 

Merging Covid State wise data with pop_merged df

******* 

In [34]:
display(us_st_cov.head(),us_st_cov.shape,merged.head(),merged.shape)

,Location,Confirmed,Cpmp,Recovered,Deaths
0,New York,"3,97,684","20,392","70,435","31,143"
1,California,"2,23,995","5,944",No data,"5,979"
2,New Jersey,"1,73,126","19,626","30,163","14,992"
3,Texas,"1,58,756","5,732","81,335","2,430"
4,Florida,"1,46,333","7,678",No data,"3,446"


(51, 5)

,Latitude,Longitude,State,StateCode,Region,Division,Fips
0,32.601011,-86.680736,Alabama,AL,South,East South Central,1
1,61.302501,-158.775020,Alaska,AK,West,Pacific,2
2,34.168219,-111.930907,Arizona,AZ,West,Mountain,4
3,34.751928,-92.131378,Arkansas,AR,South,West South Central,5
4,37.271875,-119.270415,California,CA,West,Pacific,6


(51, 7)

We have missing values in 7 rows missing values ,ignore for now

In [35]:
us_st_cov = us_st_cov[us_st_cov.Location.isin(pop_merged.State)] #use covid data for sates in pop_merged

In [36]:
final_df= pd.merge(us_st_cov, pop_merged, left_on= "Location", right_on = "State",how ="inner").drop(columns=["Location","Cpmp"])
final_df.head()

,Confirmed,Recovered,Deaths,Latitude,Longitude,State,StateCode,Region,Division,Fips,Pop_Rank,Est_Pop_2019,Perc_of_Tot_Pop
0,"3,97,684","70,435","31,143",40.705626,-73.979680,New York,NY,Northeast,Middle Atlantic,36,4,19453561,0.0586
1,"2,23,995",No data,"5,979",37.271875,-119.270415,California,CA,West,Pacific,6,1,39512223,0.1191
2,"1,73,126","30,163","14,992",40.143006,-74.731116,New Jersey,NJ,Northeast,Middle Atlantic,34,11,8882190,0.0268
3,"1,58,756","81,335","2,430",31.169336,-100.076842,Texas,TX,South,West South Central,48,2,28995881,0.0874
4,"1,46,333",No data,"3,446",27.975728,-83.833017,Florida,FL,South,South Atlantic,12,3,21477737,0.0647


In [37]:
"District of Columbia" in list(final_df.State.unique())

True

In [38]:
# to_csv

path = path = r"C:\Users\LOCALGHOST\Desktop\DESKTOP\DATASETS\AIR_QUALITY\WAQI\Processed\USA\merged_final.csv"
final_df.to_csv(path, index= False)